### Kaggle link: https://www.kaggle.com/code/noobhocai/train-stage-2

In [1]:
!pip install rank_bm25 pandarallel gensim --q

In [2]:
import os
import json
import pandas as pd
import numpy as np
import json, pickle
from rank_bm25 import BM25Okapi
import argparse
import gc
from tqdm.auto import tqdm
tqdm.pandas()
from glob import glob 
import re 
from nltk import word_tokenize as lib_tokenizer 
import string
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity

In [4]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers=10)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
df_wiki = pd.read_json("/kaggle/input/e2eqa-wiki-zalo-ai/wikipedia_20220620_cleaned/wikipedia_20220620_cleaned.jsonl", lines=True)

In [6]:
df_wiki.head()

,id,url,title,text,timestamp,revid
0,2,https://vi.wikipedia.org/wiki?curid=2,Trang Chính,"Trang Chính\n\n<templatestyles src=""Wiki2021/s...",2022-05-12 12:46:53+00:00,68591979
1,4,https://vi.wikipedia.org/wiki?curid=4,Internet Society,Internet Society\n\nInternet Society hay ISOC ...,2022-01-20 07:59:10+00:00,67988747
2,13,https://vi.wikipedia.org/wiki?curid=13,Tiếng Việt,"Tiếng Việt\n\nTiếng Việt, cũng gọi là tiếng Vi...",2022-05-29 03:42:42+00:00,68660631
3,24,https://vi.wikipedia.org/wiki?curid=24,Ohio,"Ohio\n\nOhio (viết tắt là OH, viết tắt cũ là O...",2022-04-17 08:15:22+00:00,68482118
4,26,https://vi.wikipedia.org/wiki?curid=26,California,"California\n\nCalifornia (phát âm như ""Ca-li-p...",2022-06-16 15:27:07+00:00,68738039


In [7]:
def post_process(x):
    x = " ".join(word_tokenize(strip_context(x))).strip()
    x = x.replace("\n"," ")
    x = "".join([i for i in x if i not in string.punctuation])
    x = " ".join(x.split()[:128])
    return x

dict_map = dict({})  
def word_tokenize(text): 
    global dict_map 
    words = text.split() 
    words_norm = [] 
    for w in words: 
        if dict_map.get(w, None) is None: 
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace('``', '"').replace("''", '"') 
        words_norm.append(dict_map[w]) 
    return words_norm 
 
def strip_answer_string(text): 
    text = text.strip() 
    while text[-1] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] != '(' and text[-1] == ')' and '(' in text: 
            break 
        if text[-1] == '"' and text[0] != '"' and text.count('"') > 1: 
            break 
        text = text[:-1].strip() 
    while text[0] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] == '"' and text[-1] != '"' and text.count('"') > 1: 
            break 
        text = text[1:].strip() 
    text = text.strip() 
    return text 
 
def strip_context(text): 
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text) 
    text = text.strip() 
    return text

def check_(x):
    x = str(x).lower()
    return (x.isnumeric() or "ngày" in x or "tháng" in x or "năm" in x)

def find_candidate_ids(x, raw_answer=None, already_added=[], topk=50):
    x = str(x)
    query = post_process(x).lower().split()
    tfidf_query = tfidf_model[dictionary.doc2bow(query)]
    scores = bm25_index[tfidf_query]
    top_n = list(np.argsort(scores)[::-1][:topk])
    top_n = [i for i in top_n if i not in already_added]
    # scores = list(scores[top_n])
    if raw_answer is not None:
        raw_answer = raw_answer.strip()
        if raw_answer in entity_dict:
            title = entity_dict[raw_answer].replace("wiki/","").replace("_"," ")
            extra_id = title2idx.get(title, -1)
            # print((raw_answer,title,extra_id, extra_id not in top_n))
            if extra_id != -1 and extra_id not in top_n:
                print(f"Add extra id {extra_id} for {raw_answer}")
                top_n.append(extra_id)
                top_n = list(set(top_n))
    scores = scores[top_n]
    return list(top_n), np.array(scores)

In [8]:
df_wiki['title_lower'] = df_wiki['title'].apply(lambda x: x.lower()).parallel_apply(post_process)
df_wiki['text_lower'] = df_wiki['text'].apply(lambda x: x.lower()).parallel_apply(post_process)

In [9]:
df_wiki.head()

,id,url,title,text,timestamp,revid,title_lower,text_lower
0,2,https://vi.wikipedia.org/wiki?curid=2,Trang Chính,"Trang Chính\n\n<templatestyles src=""Wiki2021/s...",2022-05-12 12:46:53+00:00,68591979,trang chính,trang chính templatestyles src wiki2021stylesc...
1,4,https://vi.wikipedia.org/wiki?curid=4,Internet Society,Internet Society\n\nInternet Society hay ISOC ...,2022-01-20 07:59:10+00:00,67988747,internet society,internet society internet society hay isoc là ...
2,13,https://vi.wikipedia.org/wiki?curid=13,Tiếng Việt,"Tiếng Việt\n\nTiếng Việt, cũng gọi là tiếng Vi...",2022-05-29 03:42:42+00:00,68660631,tiếng việt,tiếng việt tiếng việt cũng gọi là tiếng việt n...
3,24,https://vi.wikipedia.org/wiki?curid=24,Ohio,"Ohio\n\nOhio (viết tắt là OH, viết tắt cũ là O...",2022-04-17 08:15:22+00:00,68482118,ohio,ohio ohio viết tắt là oh viết tắt cũ là o là m...
4,26,https://vi.wikipedia.org/wiki?curid=26,California,"California\n\nCalifornia (phát âm như ""Ca-li-p...",2022-06-16 15:27:07+00:00,68738039,california,california california phát âm như caliphótnia ...


In [10]:
title2idx = dict([(x.strip(),y) for x,y in zip(df_wiki.title, df_wiki.index.values)])
train = json.load(open("/kaggle/input/e2eqa-wiki-zalo-ai/processed/zac2022_train_merged_final.json"))
entity_dict =  json.load(open("/kaggle/input/e2eqa-wiki-zalo-ai/processed/entities.json"))

In [19]:
!mkdir /kaggle/working/bm25_stage2
!mkdir /kaggle/working/bm25_stage2/full_text
!mkdir /kaggle/working/bm25_stage2/title

mkdir: cannot create directory ‘/kaggle/working/bm25_stage2’: File exists
mkdir: cannot create directory ‘/kaggle/working/bm25_stage2/full_text’: File exists


In [18]:
corpus = [doc.split() for doc in df_wiki['text_lower']] #simple tokenier
dictionary = Dictionary(corpus)
bm25_model = OkapiBM25Model(dictionary=dictionary)
bm25_corpus = bm25_model[list(map(dictionary.doc2bow, corpus))]
bm25_index = SparseMatrixSimilarity(bm25_corpus, num_docs=len(corpus), num_terms=len(dictionary),normalize_queries=False, normalize_documents=False)
tfidf_model = TfidfModel(dictionary=dictionary, smartirs='bnn')  # Enforce binary weighting of queries
dictionary.save("/kaggle/working/bm25_stage2/full_text/dict")
tfidf_model.save("/kaggle/working/bm25_stage2/full_text/tfidf")
bm25_index.save("/kaggle/working/bm25_stage2/full_text/bm25_index")

In [22]:
corpus = [doc.split() for doc in df_wiki['title_lower']] #simple tokenier
dictionary = Dictionary(corpus)
bm25_model = OkapiBM25Model(dictionary=dictionary)
bm25_corpus = bm25_model[list(map(dictionary.doc2bow, corpus))]
bm25_index = SparseMatrixSimilarity(bm25_corpus, num_docs=len(corpus), num_terms=len(dictionary),normalize_queries=False, normalize_documents=False)
tfidf_model = TfidfModel(dictionary=dictionary, smartirs='bnn')  # Enforce binary weighting of queries
dictionary.save("/kaggle/working/bm25_stage2/title/dict")
tfidf_model.save("/kaggle/working/bm25_stage2/title/tfidf")
bm25_index.save("/kaggle/working/bm25_stage2/title/bm25_index")